# Introduction to Machine Learning and Deep Learning
### Recurrent neural networks (RNNs)

## Introduction

A particular challenge with sequential data and modelling tasks is that the sequence lengths can vary from one dataset example to the next. This makes the use of a fixed input size architecture such as the MLP unsuitable. In addition, there can be many different types of sequential modelling tasks that we might want to consider, each of which could have different architectural requirements. 

Typical sequence modelling tasks could include:

* Text sentiment analysis (many-to-one)
* Image captioning (one-to-many)
* Language translation (many-to-many)
* Part-of-speech tagging (many-to-many)

Recurrent neural networks ([Rumelhart et al 1986b](#Rumelhart86b)) are designed to handle this variability of data lengths and diversity of problem tasks.

#### Basic RNN computation

Let $\{\mathbf{x}_t\}_{t=1}^T$ be an example sequence input, with each $\mathbf{x}_t\in\mathbb{R}^D$. Suppose that we are in the many-to-many setting, and there is a corresponding sequence of labels $\{{y}_t\}_{t=1}^T$, with $y_t\in Y$, where $Y$ could be $\{0, 1\}$ for a binary classification task for example.

The basic RNN computation is given as follows:

$$
\begin{align}
\left.
\begin{array}{rcl}
\mathbf{h}^{(1)}_t &=& \sigma\left( \mathbf{W}_{hh}^{(1)}\mathbf{h}^{(1)}_{t-1} + \mathbf{W}_{xh}^{(1)}\mathbf{x}_t + \mathbf{b}^{(1)}_h \right),\\
\hat{\mathbf{y}}_t &=& \sigma_{out}\left( \mathbf{W}_{hy}\mathbf{h}_t^{(1)} + \mathbf{b}_{y} \right),
\end{array}
\right\}\tag{1}
\end{align}
$$

for $t=1,\ldots,T$, where $\mathbf{h}^{(1)}\in\mathbb{R}^{n_1}$,
$\mathbf{W}^{(1)}_{hh}\in\mathbb{R}^{n_1\times n_1}$, $\mathbf{W}^{(1)}_{xh}\in\mathbb{R}^{n_1\times D}$, $\mathbf{b}^{(1)}_h\in\mathbb{R}^{n_{1}}$, $\mathbf{W}_{hy}\in\mathbb{R}^{n_y\times n_1}$, $\mathbf{b}_y\in\mathbb{R}^{n_{y}}$,  $\sigma$ and $\sigma_{out}$ are activation functions, $n_1$ is the number of units in the hidden layer, and $n_y$ is the dimension of the output space $Y$.

Recurrent neural networks make use of weight sharing, similar to convolutional neural networks, but this time the weights are shared across time. This allows the RNN to be 'unrolled' for as many time steps as there are in the data input $\mathbf{x}$.

The RNN also has a **persistent state**, in the form of the hidden layer $\mathbf{h}^{(1)}$. This hidden state can carry information over an arbitrary number of time steps, and so predictions at a given time step $t$ can depend on events that occurred at any point in the past, at least in principle. As with MLPs, the hidden state stores **distributed representations** of information, which allows them to store a lot of information, in contrast to hidden Markov models.

Note that the computation $(1)$ requires an **initial hidden state** $\mathbf{h}^{(1)}_0$ to be defined. In practice, this is often just set to the zero vector, although it can also be learned as additional parameters.

In Keras, the RNN is available as the layer `SimpleRNN` in the `keras.layers` module (see [the docs](https://keras.io/api/layers/recurrent_layers/simple_rnn/)). It can be included in the list of layers passed to the `Sequential` constructor, or using the functional API.

In [ ]:
import keras
from keras import ops

In [ ]:
from keras.models import Sequential
from keras.layers import Input, SimpleRNN

rnn_model = Sequential([
    Input(shape=(10, 2)),
    SimpleRNN(32, activation='tanh')  # 'tanh' is the default activation
])
rnn_model.summary()

#### Stacked RNNs
RNNs can also be made more powerful by stacking recurrent layers on top of each other:

$$
\begin{align}
\left.
\begin{array}{rcl}
\mathbf{h}^{(k)}_t &=& \sigma\left( \mathbf{W}_{hh}^{(k)}\mathbf{h}^{(k)}_{t-1} + \mathbf{W}_{xh}^{(k)}\mathbf{h}^{(k-1)}_{t} + \mathbf{b}^{(k)}_h \right),\quad k=1,\ldots, L,\\
\hat{\mathbf{y}}_t &=& \sigma_{out}\left( \mathbf{W}_{hy}\mathbf{h}^{(L)} + \mathbf{b}_{y} \right),
\end{array}
\right\}\tag{2}
\end{align}
$$

where $\mathbf{h}^{(k)}\in\mathbb{R}^{n_k}$,
$\mathbf{W}^{(k)}_{hh}\in\mathbb{R}^{n_k\times n_k}$, $\mathbf{W}^{(k)}_{xh}\in\mathbb{R}^{n_k\times n_{k-1}}$, $\mathbf{b}^{(k)}_h\in\mathbb{R}^{n_{k}}$, $\mathbf{W}_{hy}\in\mathbb{R}^{n_y\times n_L}$, $\mathbf{b}_y\in\mathbb{R}^{n_{y}}$, and we have set $n_{L+1}=n_y$, $n_0=D$, and $\mathbf{h}^{(0)} = \mathbf{x}_t$.

To create a stacked RNN in Keras, we need to obtain the full sequence of hidden states in the lower layer. This can be done using the `return_sequences` keyword argument in the layer constructor.

In [ ]:
rnn_layer_1 = SimpleRNN(16, return_sequences=True)
rnn_layer_2 = SimpleRNN(8)

In [ ]:
stacked_rnn_model = Sequential([
    Input(shape=(32, 5)),
    rnn_layer_1,
    rnn_layer_2
])
stacked_rnn_model.summary()

#### Bidirectional RNNs
Standard recurrent neural networks are uni-directional; that is, they only take past context into account. In some applications (where the full input sequence is available to make predictions) it is possible and desirable for the network to take both past and future context into account. 

For example, consider a part-of-speech (POS) tagging problem, where the task is to label each word in a sentence according to its particular part of speech, e.g. noun, adjective, verb etc. In some cases the correct label can be ambiguous given only the past context, for example the word `light` in the sentence `"There's a light ..."` could be a noun or a verb depending on how the sentence continues (e.g. `"There's a light on upstairs"` or `"There's a light breeze"`).

Bidirectional RNNs ([Schuster & Paliwal 1997](#schuster97)) are designed to look at both future and past context. They consist of two RNNs running forward and backwards in time, whose states are combined in sum way (e.g. adding or concatenating) to produce the final hidden state of the layer. 

Bidirectional recurrent neural networks (BRNNs) are implemented in Keras using the `Bidirectional` wrapper (see [the docs](https://keras.io/api/layers/recurrent_layers/bidirectional/)):

In [ ]:
from keras.layers import Bidirectional

brnn_model = Sequential([
    Input(shape=(64, 7)),
    Bidirectional(SimpleRNN(16, return_sequences=True), merge_mode='concat')
])
brnn_model.summary()

The `Bidirectional` wrapper constructs two RNNs running in different time directions. The `merge_mode='concat'` setting is the default for the `Bidirectional` constructor, and means that the bidirectional layer concatenates the hidden states from the forward and backward RNNs. This means that the number of units per time step in the output of the layer is $2\times 16 = 32$:

## Long Short Term Memory (LSTM)

As mentioned previously, recurrent neural networks can in principle use information from events that occurred many time steps earlier to make predictions at the current time step. However, in practice RNNs struggle to make use of long-term dependencies in the data due to the **vanishing gradients problem**. Gradients can explode or vanish with a large number of layers, or a large number of time steps. This problem was pointed out by [Hochreiter](#Hochreiter91), and is particularly bad in the case of RNNs, where the length of sequences can be long (e.g. 100 time steps).

The Long Short Term Memory (LSTM) network was introduced by [Hochreiter and Schmidhuber](#Hochreiter97) (and later updated by [Gers](#Gers99)) to mitigate the effect of vanishing gradients and allow the recurrent neural network to remember things for a long time.

The LSTM has inputs $\mathbf{x}_t\in\mathbb{R}^{n_{k-1}}$ and $\mathbf{h}_{t-1}\in\mathbb{R}^{n_{k}}$ just as regular RNNs. However, it also includes an internal **cell state** $\mathbf{c}_t\in\mathbb{R}^{n_{k}}$ that allows the unit to store and retain information (we drop the superscript $(k)$ in this section to ease notation). 

The LSTM cell works with a gating mechanism, consisting of logistic and linear units with multiplicative interactions. Information is allowed into the cell state when the 'write' gate is on, it can choose to erase information in the cell state when the 'forget' gate is on, and can read information from the cell state when the 'read' gate is on.

There is no neural network layer that operates directly on the cell state. This means that information is more freely able to travel across time steps in the cell state. The role of the hidden state is to manage the information flow in and out of the cell state, according to the signals provided in the inputs $\mathbf{h}_{t-1}$ and $\mathbf{x}_t$. 

#### The forget gate
The forget gate determines what information should be erased from the cell state.

The information is controlled by signals in the inputs $\mathbf{h}_{t-1}$ and $\mathbf{x}_t$ according to the following equation:

$$
\mathbf{f}_t = \sigma \left( \mathbf{W}_{f}\cdot [\mathbf{x}_t, \mathbf{h}_{t-1}] + \mathbf{b}_{f}\right),
$$

where $[\mathbf{x}_t, \mathbf{h}_{t-1}]\in\mathbb{R}^{n_k + n_{k-1}}$ is the concatenation of $\mathbf{x}_t$ and $\mathbf{h}_{t-1}$, $\mathbf{W}_{f}\in\mathbb{R}^{n_k \times (n_k + n_{k-1})}$, $\mathbf{b}_{f}\in\mathbb{R}^{n_k}$ and $\sigma$ is the sigmoid activation function. Note that entries of $\mathbf{f}_t$ will be close to one for large positive pre-activation values, and close to zero for large negative pre-activation values. The cell state is then updated

$$
\mathbf{c}_t \leftarrow \mathbf{f}_t \odot \mathbf{c}_{t-1}
$$

where $\odot$ is the Hadamard (element-wise) product, so that selected entries of the cell state $\mathbf{c}_{t-1}$ are erased, while others are retained.

#### The input and content gates
The input gate determines when information should be written into the cell state. The content gate contains the information to be written.

The input and content gates are a combination of sigmoid and tanh activation gates:

$$
\begin{align}
\mathbf{i}_t &= \sigma \left( \mathbf{W}_{i}\cdot [\mathbf{x}_t, \mathbf{h}_{t-1}] + \mathbf{b}_{i}\right)\\
\mathbf{\tilde{c}}_t &= \tanh\left( \mathbf{W}_{c}\cdot [\mathbf{x}_t, \mathbf{h}_{t-1}] + \mathbf{b}_c\right),
\end{align}
$$

where $\mathbf{W}_{i}, \mathbf{W}_c\in\mathbb{R}^{n_k \times (n_k + n_{k-1})}$ and $\mathbf{b}_i, \mathbf{b}_c\in\mathbb{R}^{n_k}$. In a similar way to the forget gate, the input gate $\mathbf{i}_t$ is used to 'zero out' selected entries in the content signal $\mathbf{\tilde{c}}_t$. The content entries that are allowed through the gate are then added into the cell state:

$$
\mathbf{c}_t \leftarrow \mathbf{c}_{t} + \mathbf{i}_t \odot \mathbf{\tilde{c}}_t
$$

#### The output gate
Finally, the output gate decides which cell state values should be output in the hidden state.

The output gate is another sigmoid gate that releases information from the cell state after passing through a tanh activation:

$$
\begin{align}
\mathbf{o}_t &= \sigma\left(\mathbf{W}_o \cdot [\mathbf{x}_t, \mathbf{h}_{t-1}] + \mathbf{b}_o\right)\\
\mathbf{h}_t &= \mathbf{o}_t \odot \tanh(\mathbf{c}_t)
\end{align}
$$

The LSTM network has been immensely successful in sequence modelling tasks, including handwriting recognition ([Graves et al 2009](#Graves09)), speech recognition ([Graves et al 2013](#Graves13)), machine translation ([Wu et al 2016](#Wu16)) and reinforcement learning for video games ([Vinyals et al 2019](#Vinyals19)).

Another type of gated recurrent cell that should be mentioned is the Gated Recurrent Unit (GRU), proposed in [Cho et al 2014](#Cho14), which simplifies the architecture by combining the forget and input gates into a single 'update' gate, and also merges the cell state and hidden state. We will not go into detail of this cell architecture, for more details refer to the paper.

In [ ]:
from keras.layers import Input, LSTM, Dense
from keras.optimizers import RMSprop

import os
import numpy as np
import matplotlib.pyplot as plt

#### HAR dataset

We will use the [Human Activity Recognition (HAR) Using Smartphones](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) dataset, which consists of the readings from an accelerometer (which measures acceleration) carried by a human doing different activities. The six activities are walking horizontally, walking upstairs, walking downstairs, sitting, standing and laying down. The accelerometer is inside a smartphone, and, every 0.02 seconds (50 times per second), it takes six readings: linear and gyroscopic acceleration in the x, y and z directions. See [this link](https://archive.ics.uci.edu/ml/datasets/Human+Activity+Recognition+Using+Smartphones) for details. The data can be downloaded at [this Drive link](https://drive.google.com/file/d/1-27FKOudyhBvpQCAGGCJjkFLf_o_tJHI/view?usp=sharing). 

- Davide Anguita, Alessandro Ghio, Luca Oneto, Xavier Parra and Jorge L. Reyes-Ortiz. A Public Domain Dataset for Human Activity Recognition Using Smartphones. 21th European Symposium on Artificial Neural Networks, Computational Intelligence and Machine Learning, ESANN 2013. Bruges, Belgium 24-26 April 2013. 

The goal is to use the accelerometer data to predict the activity.

In [ ]:
# Load the HAR data

x_train = np.load(os.path.join('data', 'HAR_x_train.npy'))
y_train = np.load(os.path.join('data', 'HAR_y_train.npy'))
x_test  = np.load(os.path.join('data', 'HAR_x_test.npy'))
y_test  = np.load(os.path.join('data', 'HAR_y_test.npy'))

In [ ]:
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
np.unique(y_train)

In [ ]:
class_names = ['Walking horizontally', 'Walking upstairs', 'Walking downstairs',
               'Sitting', 'Standing', 'Laying']

In [ ]:
# Plot some data examples

for label in range(6):
    x_label = x_train[y_train == label]
    fig, ax = plt.subplots(figsize=(10, 1))
    i = np.random.choice(x_label.shape[0])
    ax.imshow(x_label[i].T, cmap='Greys', vmin=-1, vmax=1)
    ax.axis('off')
    ax.set_title(class_names[label])
    plt.show()

In [ ]:
y_train_oh = ops.one_hot(y_train, num_classes=len(class_names))
y_test_oh = ops.one_hot(y_test, num_classes=len(class_names))

In [ ]:
y_train_oh.shape

In [ ]:
model = Sequential([
    Input(shape=[None, 6]),
    LSTM(32, return_sequences=True),
    LSTM(32),
    Dense(6, activation='softmax')
])
model.summary()

In [ ]:
model.compile(optimizer=RMSprop(learning_rate=0.005), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train_oh, epochs=10, batch_size=32, validation_data=(x_test, y_test_oh))

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 3))
ax1.plot(history.history['loss'], label='train')
ax1.plot(history.history['val_loss'], label='val')
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss")
ax1.legend()
ax2.plot(history.history['accuracy'], label='train')
ax2.plot(history.history['val_accuracy'], label='val')
ax2.set_xlabel("Epoch")
ax2.set_ylabel("Accuracy")
ax2.legend()
plt.show()

In [ ]:
preds = model.predict(x_test)

In [ ]:
wrong_inx = np.where(np.argmax(preds, axis=-1) != y_test)[0]

In [ ]:
num_samples = 5

for i in np.random.choice(wrong_inx, num_samples, replace=False):
    fig, ax = plt.subplots(figsize=(10, 1))    
    ax.imshow(x_test[i].T, cmap='Greys', vmin=-1, vmax=1)
    ax.axis('off')
    ax.set_title(f"True label: {class_names[y_test[i]]}, predicted: {class_names[np.argmax(preds[i])]}")
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm = confusion_matrix(y_test, np.argmax(preds, axis=-1))
ConfusionMatrixDisplay(cm, display_labels=class_names).plot(xticks_rotation='vertical')

## References

<a class="anchor" id="Cho14"></a>
* Cho, K., van Merriënboer, B., Gülçehre, Ç., Bahdanau, D., Bougares, F., Schwenk, H., & Bengio, Y. (2014), "Learning phrase representations using rnn encoder–decoder for statistical machine translation", in *Proceedings of the 2014 Conference on Empirical Methods in Natural Language Processing (EMNLP)*, 1724–1734.
<a class="anchor" id="Gers99"></a>
* Gers, F.A. (1999), "Learning to forget: Continual prediction with LSTM", *9th International Conference on Artificial Neural Networks: ICANN '99*, 850–855.
<a class="anchor" id="Graves09"></a>
* Graves, A., Liwicki, M., Fernández, S., Bertolami, R., Bunke, H., & Schmidhuber, J. (2009), "A Novel Connectionist System for Unconstrained Handwriting Recognition", *IEEE Transactions on Pattern Analysis and Machine Intelligence*, **31** (5), 855–868.
<a class="anchor" id="Graves13"></a>
* Graves, A., Mohamed, A.-R., Hinton, G. (2013), "Speech Recognition with Deep Recurrent Neural Networks", arXiv preprint, abs/1303.5778.
<a class="anchor" id="Hochreiter91"></a>
* Hochreiter, S. (1991), "Untersuchungen zu dynamischen neuronalen Netzen", Diploma thesis, Institut für Informatik, Lehrstuhl Prof. Brauer, Technische Universität München.
<a class="anchor" id="Hochreiter97"></a>
* Hochreiter, S. and Schmidhuber, J. (1997), "Long short-term memory", *Neural Computation*, **9** (8), 1735–1780.
<a class="anchor" id="Rumelhart86b"></a>
* Rumelhart, D. E., Hinton, G., and Williams, R. (1986b), "Learning representations by back-propagating errors", Nature, **323**, 533-536.
<a class="anchor" id="Schuster97"></a>
* Schuster, M. & Paliwal, K. K. (1997), "Bidirectional Recurrent Neural Networks", *IEEE Transactions on Signal Processing*, **45** (11), 2673-2681.
<a class="anchor" id="Vinyals19"></a>
* Vinyals, O., Babuschkin, I., Czarnecki, W. M., Mathieu, M., Dudzik, A., Chung, J., Choi, D. H., Powell, R., Ewalds, T., Georgiev, P., Oh, J., Horgan, D., Kroiss, M., Danihelka, I., Huang, A., Sifre, L., Cai, T., Agapiou, J. P., Jaderberg, M., Vezhnevets, A. S., Leblond, R., Pohlen, T., Dalibard, V., Budden, D., Sulsky, Y., Molloy, J., Paine, T. L., Gulcehre, C., Wang, Z., Pfaff, T., Wu, Y., Ring, R., Yogatama, D., Wünsch, D., McKinney, K., Smith, O., Schaul, T., Lillicrap, T., Kavukcuoglu, K., Hassabis, D., Apps, C., & Silver, D.(2019) "Grandmaster level in StarCraft II using multi-agent reinforcement learning", *Nature*, **575** (7782), 350-354.
<a class="anchor" id="Wu16"></a>
* Wu, Y., Schuster, M., Chen, Z., Le, Q. V., Norouzi, M, Macherey, W., Krikun, M., Cao, Y., & Gao, Q. (2016), "Google's Neural Machine Translation System: Bridging the Gap between Human and Machine Translation", arXiv preprint, abs/1609.08144.